In [1]:
import lightgbm as lgb
from catboost import CatBoostRegressor
import os, glob
import json
import folium
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error
from math import sqrt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split



import warnings
# Filter out specific ValueWarnings from statsmodels
warnings.filterwarnings("ignore")

In [2]:
# !ls /kaggle/input/

In [3]:
ndays=1
exps_dir = "../../exps"
if os.path.exists(exps_dir) == False: 
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/exp"
os.makedirs(save_dir, exist_ok=True)

In [4]:
df_train_features=pd.read_csv(f'{save_dir}/train_lag{ndays}.csv')
df_train_features

,row_id,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,2018347,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,2018348,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,2018349,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,2018350,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [5]:
df_train_features

,row_id,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000
1,366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226
2,366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000
3,366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899
4,366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,2018347,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233
1651898,2018348,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000
1651899,2018349,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404
1651900,2018350,15,1,3,0,60,2023-05-31,29.0,34.0,2023-05-29,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,0.000


In [6]:
numeric_columns=(list(df_train_features.select_dtypes('number')))[0:len(list(df_train_features.select_dtypes('number')))-10]
numeric_columns.append('target')
numeric_columns.remove('row_id')
numeric_columns.remove('data_block_id')
numeric_columns.remove('data_block_id_right')
numeric_columns.remove('hours_ahead')
numeric_columns.remove('hours_ahead_fl_7d')
numeric_columns.remove('county')
numeric_columns.remove('product_type')
numeric_columns.remove('is_consumption')
numeric_columns.remove('prediction_unit_id')


numeric_columns

['is_business',
 'lowest_price_per_mwh',
 'highest_price_per_mwh',
 'eic_count',
 'installed_capacity',
 'euros_per_mwh',
 'temperature',
 'dewpoint',
 'cloudcover_high',
 'cloudcover_low',
 'cloudcover_mid',
 'cloudcover_total',
 '10_metre_u_wind_component',
 '10_metre_v_wind_component',
 'direct_solar_radiation',
 'surface_solar_radiation_downwards',
 'snowfall',
 'total_precipitation',
 'hours_ahead_fl',
 'temperature_fl',
 'dewpoint_fl',
 'cloudcover_high_fl',
 'cloudcover_low_fl',
 'cloudcover_mid_fl',
 'cloudcover_total_fl',
 '10_metre_u_wind_component_fl',
 '10_metre_v_wind_component_fl',
 'direct_solar_radiation_fl',
 'surface_solar_radiation_downwards_fl',
 'snowfall_fl',
 'total_precipitation_fl',
 'hours_ahead_fd_7d',
 'temperature_fd_7d',
 'dewpoint_fd_7d',
 'cloudcover_high_fd_7d',
 'cloudcover_low_fd_7d',
 'cloudcover_mid_fd_7d',
 'cloudcover_total_fd_7d',
 '10_metre_u_wind_component_fd_7d',
 '10_metre_v_wind_component_fd_7d',
 'direct_solar_radiation_fd_7d',
 'surface_so

In [7]:
category_columns=list(df_train_features.select_dtypes('category'))
category_columns.append('data_block_id')
category_columns.append('data_block_id_right')
category_columns.append('hours_ahead')
category_columns.append('hours_ahead_fl_7d')
category_columns.append('county')
category_columns.append('row_id')
category_columns.append('product_type')
category_columns.append('is_consumption')
category_columns.append('prediction_unit_id')

In [8]:
# label_encoders = {}
# encoder = OneHotEncoder(sparse_output=False)


# one_hot_encoded = encoder.fit_transform(df_train_features[category_columns])

# one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(category_columns))
# df_train_features = pd.concat([df_train_features, one_hot_df], axis=1)
# df_train_features = df_train_features.drop(category_columns, axis=1)





In [ ]:
df_train,df_valid = train_test_split(df_train_features, test_size=0.33, random_state=42, shuffle=False)
Y_train = df_train['target']
Y_valid = df_valid['target']

In [9]:
# for column in category_columns:
#     label_encoder = LabelEncoder()
#     df_train_features[column] = label_encoder.fit_transform(df_train_features[column])
#     df_train_features[column] = label_encoder  

In [10]:
scaler = StandardScaler()
df_train[numeric_columns] = scaler.fit_transform(df_train[numeric_columns])
df_train

,row_id,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,366048,0,-1.088528,1,0,0,2022-01-01,-0.423582,-0.380074,2021-12-30,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,-0.306737
1,366049,0,-1.088528,1,1,0,2022-01-01,-0.423582,-0.380074,2021-12-30,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,0.169972
2,366050,0,-1.088528,2,0,1,2022-01-01,-0.423582,-0.380074,2021-12-30,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,-0.306737
3,366051,0,-1.088528,2,1,1,2022-01-01,-0.423582,-0.380074,2021-12-30,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,-0.258337
4,366052,0,-1.088528,3,0,2,2022-01-01,-0.423582,-0.380074,2021-12-30,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,-0.306720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,2018347,15,0.918672,0,1,64,2023-05-31,-1.439088,-1.427649,2023-05-29,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,-0.094124
1651898,2018348,15,0.918672,1,0,59,2023-05-31,-1.439088,-1.427649,2023-05-29,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,-0.306737
1651899,2018349,15,0.918672,1,1,59,2023-05-31,-1.439088,-1.427649,2023-05-29,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,-0.276118
1651900,2018350,15,0.918672,3,0,60,2023-05-31,-1.439088,-1.427649,2023-05-29,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,-0.306737


In [ ]:
df_valid[numeric_columns] = scaler.fit_transform(df_valid[numeric_columns])
df_valid

In [11]:
df_train.drop('row_id',axis=1)
df_valid.drop('row_id',axis=1)

,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,...,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target
0,0,-1.088528,1,0,0,2022-01-01,-0.423582,-0.380074,2021-12-30,122.0,...,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,-0.306737
1,0,-1.088528,1,1,0,2022-01-01,-0.423582,-0.380074,2021-12-30,122.0,...,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,0.169972
2,0,-1.088528,2,0,1,2022-01-01,-0.423582,-0.380074,2021-12-30,122.0,...,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,-0.306737
3,0,-1.088528,2,1,1,2022-01-01,-0.423582,-0.380074,2021-12-30,122.0,...,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,-0.258337
4,0,-1.088528,3,0,2,2022-01-01,-0.423582,-0.380074,2021-12-30,122.0,...,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,-0.306720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651897,15,0.918672,0,1,64,2023-05-31,-1.439088,-1.427649,2023-05-29,637.0,...,31,3,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,-0.094124
1651898,15,0.918672,1,0,59,2023-05-31,-1.439088,-1.427649,2023-05-29,637.0,...,31,3,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,-0.306737
1651899,15,0.918672,1,1,59,2023-05-31,-1.439088,-1.427649,2023-05-29,637.0,...,31,3,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,-0.276118
1651900,15,0.918672,3,0,60,2023-05-31,-1.439088,-1.427649,2023-05-29,637.0,...,31,3,5,2023,15_1_3_0,0.522133,-0.852864,-0.258819,0.965926,-0.306737


In [12]:
df_train_features.to_csv(f'{save_dir}/train_lag{ndays}_pre_processing.csv')
df_train.to_csv(f'{save_dir}/df_train{ndays}_pre_processing.csv')
df_valid.to_csv(f'{save_dir}/df_valid{ndays}_pre_processing.csv')